<a href="https://colab.research.google.com/github/MathewBiddle/bio_data_guide/blob/la_mbon_benthic_core/datasets/la_mbon_benthic_core/LA_MBON_Benthic_Core_Samples_2_DarwinCore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data from ERDDAP and convert to Darwin Core

https://gcoos4.geos.tamu.edu/erddap/tabledap/mbon_la_2023.html

https://github.com/ioos/mbon_data_workshop_2025/issues/9

In [2]:
!pip install erddapy
!pip install pyworms

## Go get data and metadata

In [3]:
from erddapy import ERDDAP
import pandas as pd

server = "https://gcoos4.geos.tamu.edu/erddap/"

protocol = "tabledap"

search_for = 'sanctsound "Sound Production"'

e = ERDDAP(server=server, protocol=protocol)

e.dataset_id = "mbon_la_2023"

meta = pd.read_csv(e.get_info_url().replace('.html','.csv'))

df = e.to_pandas()

df.sample(n=5)

,basin,site,replicate,latitude (degrees_north),longitude (degrees_east),date,time2,sample_method,weather,wind (knots),...,Amphioplus_sp (count),Microphiopholis_atra (count),Pentamera_pulcherrima (count),Cnidaria (count),Anemone (count),Leech (count),Bug (count),Ephemeroptera_sp (count),Eel (count),Copepod (count)
201,NaN,BB1,R3,29.390940,-89.924710,7/12/2023,10:43,push_core,sunny,4,...,0,0,0,0,0,0,0,0,0,0
205,NaN,BB2,R1,29.338230,-89.929030,7/12/2023,12:18,push_core,sunny,2,...,0,1,0,0,0,0,0,0,0,0
366,NaN,AD2,R1_sub2,29.422956,-91.279755,7/3/2023,13:43,push_core,partly_cloudy,1,...,0,0,0,0,0,0,0,0,0,0
348,NaN,MD5,R6,29.324260,-89.403500,7/16/2023,16:20,push_core,sunny,1,...,0,0,0,0,0,0,0,0,0,0
170,Atchafalaya,AD3,R4,29.490540,-91.339640,4/14/2023,13:25,push_core,sunny,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
meta.sample(n=5)

,Row Type,Variable Name,Attribute Name,Data Type,Value
161,attribute,depth,standard_name,String,depth
427,attribute,Leiocapitella_sp_B,units,String,count
365,variable,Polydora_aggregata,NaN,int,NaN
411,attribute,Notomastus_daueri,long_name,String,Notomastus Daueri
861,attribute,Anemone,ioos_category,String,Unknown


# convert wide to long format

Right now, species are in the column headers. We need to make a row for each species.

In [42]:
non_species = [col for col in df.columns if "(count)" not in str(col)]

df_melt = pd.melt(df, id_vars=non_species, var_name='species')

df_melt.sample(n=5)

,basin,site,replicate,latitude (degrees_north),longitude (degrees_east),date,time2,sample_method,weather,wind (knots),...,pH,temp (degrees Celsius),sal (psu),bott_chlor (ug/L),surf_chlor (ug/L),depth (m),sediment,high_organic,species,value
1707,Terrebonne,TB6,R3,29.31804,-90.34680,4/10/2023,10:22,push_core,partly_cloudy,4,...,7.91,21.327,19.88,11.41,11.77,2.10312,clay,no,Tellinidae_sp (count),0.0
38696,NaN,BB4,R1,29.29560,-89.91075,7/12/2023,13:11,push_core,partly_cloudy,2,...,7.99,31.090,30.20,16.11,7.61,0.91440,clay,no,Microphiopholis_atra (count),0.0
21528,Terrebonne,TB1,R3,29.25336,-90.66844,4/15/2023,0:00,push_core,mostly_cloudy,2,...,7.82,26.895,17.34,23.58,26.19,1.06680,muddy,no,Magelona_sp_I (count),0.0
13979,NaN,BB2,R3,29.33185,-89.92684,7/12/2023,12:00,push_core,sunny,3,...,7.93,30.620,31.18,7.51,10.00,2.19456,sandy,no,Notomastus_daueri (count),0.0
13772,Barataria,BB1,R1_sub3,29.39825,-89.92838,4/20/2023,11:04,push_core,sunny,4,...,8.36,22.003,12.76,20.88,18.75,2.43840,muddy,no,Notomastus_daueri (count),0.0


# only find where species was observed

We don't care about the value=0 data

In [43]:
df_melt = df_melt.loc[df_melt['value']>0]

df_melt.sample(n=5)

,basin,site,replicate,latitude (degrees_north),longitude (degrees_east),date,time2,sample_method,weather,wind (knots),...,pH,temp (degrees Celsius),sal (psu),bott_chlor (ug/L),surf_chlor (ug/L),depth (m),sediment,high_organic,species,value
8386,NaN,TB6,R1_sub2,29.31097,-90.34413,7/5/2023,13:18,push_core,partly_cloudy,1,...,7.84,32.170,21.81,24.08,23.13,1.40000,muddy,yes,Arcuatula_papyria (count),2.0
3325,Terrebonne,TB6,R1,29.31097,-90.34413,4/10/2023,10:30,push_core,partly_cloudy,4,...,7.93,21.268,19.75,4.40,12.85,1.98120,clay,no,Austromacoma_constricta (count),1.0
10413,NaN,TB6,R2,29.31559,-90.33950,7/5/2023,13:01,ponar_grab,partly_cloudy,1,...,7.85,31.980,21.64,24.20,18.25,2.00000,shelly,no,Manayunkia_aestuarina (count),2.0
16609,Barataria,BB1,R3,29.39094,-89.92471,4/20/2023,10:59,ponar_grab,sunny,4,...,8.37,21.974,13.44,23.68,20.94,2.40790,sandy,no,Glycinde_multidens (count),5.0
8667,Atchafalaya,AD2,R4,29.42220,-91.28162,4/14/2023,11:25,push_core,sunny,1,...,7.67,17.927,0.10,5.60,4.52,1.88976,silty,no,Alitta_succinea (count),1.0


Clean up the species names

In [44]:
df_melt['species'] = df_melt['species'].str.replace(" (count)","")

df_melt['species'].unique()

array(['Nemerteans', 'Oligochaetes', 'Chironomidae_sp', 'Bivalve',
       'Tellinidae_sp', 'Ameritella_versicolor', 'Psammotreta_brevifrons',
       'Macoploma_tenta', 'Austromacoma_constricta', 'Mulinia_lateralis',
       'Spisula_raveneli', 'Rangia_sp', 'Rangia_cuneata',
       'Dosinia_discus', 'Agriopoma_texasianum',
       'Mercenaria_campechiensis', 'Tagelus_divisus',
       'Kelliopsis_elevata', 'Nuculana_concentrica', 'Ensis_minor',
       'Arcuatula_papyria', 'Alitta_succinea', 'Nereis_splendida',
       'Ceratocephale_oculata', 'Kirkegaardia_annulosa',
       'Manayunkia_aestuarina', 'Acromegalomma_bioculatum',
       'Paraprionospio_pinnata', 'Polydora_aggregata', 'Dispio_uncinata',
       'Boccardiella_sp_A', 'Capitellidae_sp', 'Mediomastus_ambiseta',
       'Capitella_capitata', 'Notomastus_daueri', 'Decamastus_gracilis',
       'Leiocapitella_sp_B', 'Mediomastus_californiensis',
       'Synelmis_albini', 'Hermundura_americana', 'Sigambra_tentaculata',
       'Glycinde_mul

Now we need to build a mapping table to WoRMS identification.

For names that have matches != 1, need to do some human deciding.


In [45]:
import pyworms

df_worms_mapping = dict()

for species in df_melt['species'].unique():
  worms_info = pyworms.aphiaRecordsByMatchNames(species, marine_only=True)
  df_worms_mapping[species] = worms_info[0]

  if len(worms_info[0]) != 1:
    print(f'Found {len(worms_info[0])} matches for {species}')

Found 3 matches for Nemerteans
Found 0 matches for Oligochaetes
Found 0 matches for Bivalve
Found 2 matches for Ensis_minor
Found 3 matches for Nereis_splendida
Found 2 matches for Glycera_sp
Found 0 matches for Allia_quadrilobata
Found 0 matches for Allia_trilobata
Found 0 matches for Cumaceae_sp
Found 0 matches for Leech
Found 0 matches for Bug
Found 0 matches for Ephemeroptera_sp
Found 0 matches for Eel
Found 0 matches for Copepod


Need to figure out what to do with those. First, insert the species information we know.

In [49]:
for species in df_worms_mapping:
  if len(df_worms_mapping[species])==1:
    df_melt.loc[df_melt['species']==species,'kingdom'] = df_worms_mapping[species][0]['kingdom']
    df_melt.loc[df_melt['species']==species,'scientificName'] = df_worms_mapping[species][0]['scientificname']
    df_melt.loc[df_melt['species']==species,'scientificNameID'] = df_worms_mapping[species][0]['lsid']
df_melt.sample(n=5)

,basin,site,replicate,latitude (degrees_north),longitude (degrees_east),date,time2,sample_method,weather,wind (knots),...,bott_chlor (ug/L),surf_chlor (ug/L),depth (m),sediment,high_organic,species,value,kingdom,scientificName,scientificNameID
14436,NaN,TB3,R1,29.059600,-90.72260,7/4/2023,11:56,push_core,partly_cloudy,1,...,8.48,6.38,3.2000,muddy,no,Decamastus_gracilis,1.0,Animalia,Decamastus gracilis,urn:lsid:marinespecies.org:taxname:326885
28742,NaN,AD5,R2,29.619553,-91.32493,7/3/2023,9:19,ponar_grab,partly_cloudy,1,...,7.56,7.99,5.1816,detritus,no,Apocorophium_louisianum,41.0,Animalia,Apocorophium louisianum,urn:lsid:marinespecies.org:taxname:421575
261,NaN,TB3,R1,29.059600,-90.72260,7/4/2023,11:56,push_core,partly_cloudy,1,...,8.48,6.38,3.2000,muddy,no,Nemerteans,8.0,NaN,NaN,NaN
33002,NaN,BB1,R1,29.398250,-89.92838,7/12/2023,11:01,push_core,sunny,4,...,12.98,9.32,2.1336,clay,no,Mysidae_sp,1.0,Animalia,Mysidae,urn:lsid:marinespecies.org:taxname:119822
11016,Terrebonne,TB5,R5,29.152510,-90.20687,4/10/2023,12:11,push_core,partly_cloudy,4,...,16.83,5.78,0.6096,clay,no,Paraprionospio_pinnata,5.0,Animalia,Paraprionospio pinnata,urn:lsid:marinespecies.org:taxname:131140


What to do with the stuff we don't know??

In [50]:
df_melt.loc[df_melt['species']=='Bug']

,basin,site,replicate,latitude (degrees_north),longitude (degrees_east),date,time2,sample_method,weather,wind (knots),...,bott_chlor (ug/L),surf_chlor (ug/L),depth (m),sediment,high_organic,species,value,kingdom,scientificName,scientificNameID
40532,Barataria,BB5,R1,29.287510,-90.004760,4/26/2023,14:01,ponar_grab,sunny,3,...,19.27,15.68,2.37744,muddy,no,Bug,1.0,NaN,NaN,NaN
40732,NaN,BB5,R2,29.288790,-89.998300,7/12/2023,14:30,push_core,cloudy,3,...,14.81,14.97,1.52400,clay,no,Bug,1.0,NaN,NaN,NaN
40782,NaN,TB5,R4,29.156240,-90.209210,7/5/2023,10:44,push_core,cloudy,1,...,10.42,11.89,0.90000,muddy,no,Bug,1.0,NaN,NaN,NaN
40885,NaN,AD4,R3,29.634586,-91.260570,7/3/2023,16:41,push_core,sunny,1,...,14.11,16.49,0.60960,muddy,no,Bug,1.0,NaN,NaN,NaN
40904,NaN,AD6,R6,29.553507,-91.325264,7/3/2023,10:20,ponar_grab,partly_cloudy,1,...,3.60,10.85,5.18160,muddy,no,Bug,1.0,NaN,NaN,NaN
